In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import spacy
import matplotlib.pyplot as plt
import time
import multiprocessing
import zipfile


base_loc = "/data/"
raw_zip_file = "raw/kaggle/coleridge_initiative/coleridgeinitiative-show-us-the-data.zip"
unzip_loc = "processed/kaggle/coleridge_initiative/"

test_folder = "test"
train_folder = "train"
train_y_file = "train.csv"
submission_file = "sample_submission.csv"

output_folder = "processing_1"


nlp = spacy.load("en_core_web_sm")

In [7]:
# raw_zip_file = "/data/raw/kaggle/coleridge_initiative/coleridgeinitiative-show-us-the-data.zip"

# with zipfile.ZipFile(os.path.join(base_loc, raw_zip_file), 'r') as zip_ref:
#     zip_ref.extractall(os.path.join(base_loc, unzip_loc))

In [2]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

In [3]:
def get_sentences(ind, base_loc, train_folder, file_id, label):
    '''
        ind: index of the file to print out the 
    '''
    
    if ind % 50 ==0:
        print("\n", ind)
    temp_file_loc = os.path.join(base_loc, train_folder, file_id+".json")
    json_data = json.load(open(temp_file_loc, "r"))
    sentences = []
    for elem in json_data:
        temp_text = elem["text"]
        temp_doc = nlp(temp_text)
        sents = list(temp_doc.sents)
        for s_ind, sent in enumerate(sents):
            sent = str(sent)
            if label in sent:
                sentences.append({
                    "file": file_id+ ".json",
                    "section": elem["section_title"],
                    "sentence_id": s_ind,
                    "sentence": sent,
                    "label": label
                })
            else:
                sentences.append({
                    "file": file_id+ ".json",
                    "section": elem["section_title"],
                    "sentence_id": s_ind,
                    "sentence": sent
                })
    return sentences

In [4]:
base_loc = os.path.join(base_loc, unzip_loc)
train_df = pd.read_csv(os.path.join(base_loc,  train_y_file))
cloumns = list(train_df.columns)

id_column = train_df["Id"]
label_column = train_df["dataset_label"]

inputs = [(i, base_loc, train_folder, l[0], l[1]) for i,l in enumerate(zip(id_column, label_column)) if i<10000]

print("processing started")
process_pool = multiprocessing.Pool(3)
output = process_pool.starmap(get_sentences, inputs)

print(len(output))
print(len(output[0]))


processing started

 0

 850

 1700

 50

 1750

 900

 100

 1800

 1850

 150

 950

 1900

 1950

 2000

 1000

 200

 2050

 1050

 2100

 250

 2150

 1100

 2200

 2250

 300

 1150

 2300

 350

 2350

 1200

 2400

 400

 1250

 2450

 450

 1300

 2500

 500

 1350

 2550

 550

 600

 2600

 1400

 650

 700

 2650

 1450

 750

 1500

 800

 2700

 3350

 1550

 2750

 1600

 3400

 1650

 2800

 2850

 4200

 3450

 2900

 3500

 4250

 2950

 4300

 3550

 3600

 3000

 4350

 3650

 3050

 4400

 3700

 4450

 3100

 3750

 4500

 3150

 3800

 4550

 3850

 3200

 4600

 3900

 4650

 3250

 3950

 4700

 4000

 3300

 4750

 4050

 4800

 4100

 5050

 4150

 4850

 5850

 5100

 4900

 5900

 5150

 4950

 5950

 5200

 5000

 6700

 6000

 5250

 6750

 5300

 6050

 5350

 6800

 6100

 5400

 6150

 6850

 5450

 6200

 6900

 5500

 6250

 5550

 6300

 6950

 5600

 6350

 6400

 5650

 7000

 6450

 5700

 6500

 5750

 7050

 6550

 5800

 6600

 7550

 7100

 6

In [5]:
output = [o1 for o in output for o1 in o]
print(len(output))
positive_sentences = [o for o in output if "label" in o]
negative_sentences = [o for o in output if "label" not in o]
print(len(positive_sentences))
print(len(negative_sentences))

2461475
37667
2423808


In [6]:
positive_output_file = os.path.join(base_loc, output_folder, "pos.json")
negative_output_file = os.path.join(base_loc, output_folder, "neg.json")

json.dump(positive_sentences, open(positive_output_file, 'w'))
json.dump(negative_sentences, open(negative_output_file, 'w'))
